In [8]:
import math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [2]:
ny_crime = pd.read_excel('crime_in_ny_state.xls', header=4, nrows=348)
ny_crime = ny_crime.rename(columns= {'City':'city', 'Population':'pop', 
                          'Violent\ncrime':'violent_crime', 
                          'Murder and\nnonnegligent\nmanslaughter': 'murder',
                          'Rape\n(revised\ndefinition)1': 'rape_new',
                          'Rape\n(legacy\ndefinition)2': 'rape_old', 
                          'Robbery': 'robbery', 
                          'Aggravated\nassault': 'assault', 
                          'Property\ncrime': 'property_crime', 
                          'Burglary': 'burglary', 
                          'Larceny-\ntheft': 'larceny', 
                          'Motor\nvehicle\ntheft':'vehicle_theft', 
                          'Arson3':'arson'})

In [3]:
ny_crime_no_ny = ny_crime.loc[ny_crime.murder < 300, ]

In [57]:
ny_crime_no_ny.head()

,city,pop,violent_crime,murder,rape_new,rape_old,robbery,assault,property_crime,burglary,larceny,vehicle_theft,arson
0,Adams Village,1861,0,0,NaN,0,0,0,12,2,10,0,0.0
1,Addison Town and Village,2577,3,0,NaN,0,0,3,24,3,20,1,0.0
2,Akron Village,2846,3,0,NaN,0,0,3,16,1,15,0,0.0
3,Albany,97956,791,8,NaN,30,227,526,4090,705,3243,142,NaN
4,Albion Village,6388,23,0,NaN,3,4,16,223,53,165,5,NaN


In [29]:
features = pd.DataFrame()
features['pop'] = ny_crime_no_ny['pop']
features['murder_cat'] = np.where(ny_crime_no_ny['murder'] >0, 1, 0)
features['robbery_cat'] = np.where(ny_crime_no_ny['robbery'] >0, 1, 0)
features['rape'] = ny_crime_no_ny['rape_old']
features['assault'] = ny_crime_no_ny['assault']


## Model 1 - Murder, Robbery and Rape

In [48]:
regr_1 = linear_model.LinearRegression()
Y_1 = ny_crime_no_ny['property_crime'].values.reshape(-1, 1)
X_1 = features[['murder_cat', 'robbery_cat', 'rape']]

#### Overall

In [62]:
regr_1.fit(X_1, Y_1)
regr_1.score(X_1, Y_1)

0.9103158817754645

#### Cross validation

In [54]:
r_sq_1 = cross_val_score(regr_1, X_1, Y_1, cv=5, scoring='explained_variance')
print('R-squareds: ', r_sq_1)
print('Mean: ', np.mean(r_sq_1))

R-squareds:  [0.93330734 0.49402076 0.48078547 0.89270774 0.94566537]
Mean:  0.7492973349359011


#### Calculate again using statsmodels

In [58]:
features['prop_crime'] = ny_crime_no_ny['property_crime']

In [59]:
linear_model_1 = 'prop_crime ~ murder_cat+robbery_cat+rape'

lm_1 = smf.ols(formula=linear_model_1, data=features).fit()

In [63]:
lm_1.rsquared ## Should be same as overall

0.9103158817754646

In [64]:
lm_1.pvalues

Intercept       2.585257e-01
murder_cat      5.400710e-03
robbery_cat     5.883712e-06
rape           1.232519e-163
dtype: float64

Intercept is not statistically significant.

## Model 2 - Murder, Robbery and Assault

In [50]:
regr_2 = linear_model.LinearRegression()
Y_2 = ny_crime_no_ny['property_crime'].values.reshape(-1, 1)
X_2 = features[['murder_cat', 'robbery_cat', 'assault']]

#### Overall

In [65]:
regr_2.fit(X_2, Y_2)
regr_2.score(X_2, Y_2)

0.8984655801628051

#### Cross validation

In [55]:
r_sq_2 = cross_val_score(regr_2, X_2, Y_2, cv=5, scoring='explained_variance')
print('R-squareds: ', r_sq_2)
print('Mean: ', np.mean(r_sq_2))

R-squareds:  [0.93932668 0.5413865  0.43857311 0.90160724 0.83289244]
Mean:  0.7307571915282283


#### Calculate again using statsmodels

In [66]:
linear_formula_2 = 'prop_crime ~ murder_cat+robbery_cat+assault'
lm_2 = smf.ols(formula=linear_formula_2, data = features).fit()

In [67]:
lm_2.rsquared

0.898465580162805

In [68]:
lm_2.pvalues

Intercept       2.828603e-01
murder_cat      1.559155e-01
robbery_cat     1.288357e-07
assault        2.177542e-154
dtype: float64

Intercept and murder are not statistically significant this time.

## Model 3 - Murder, Robbery and Population

In [52]:
regr_3 = linear_model.LinearRegression()
Y_3 = ny_crime_no_ny['property_crime'].values.reshape(-1, 1)
X_3 = features[['murder_cat', 'robbery_cat', 'pop']]

#### Overall

In [69]:
regr_3.fit(X_3, Y_3)
regr_3.score(X_3, Y_3)

0.7889428000907828

#### Cross validation

In [56]:
r_sq_3 = cross_val_score(regr_3, X_3, Y_3, cv=5, scoring='explained_variance')
print('R-squareds: ', r_sq_3)
print('Mean: ', np.mean(r_sq_3))

R-squareds:  [0.82903036 0.44692663 0.40281642 0.80032172 0.46571   ]
Mean:  0.5889610249048312


#### Calculate using statsmodels

In [70]:
linear_formula_3 = 'prop_crime ~ murder_cat+robbery_cat+pop'

lm_3 = smf.ols(formula=linear_formula_3, data=features).fit()

In [71]:
lm_3.rsquared

0.7889428000907828

In [72]:
lm_3.pvalues

Intercept       8.205933e-03
murder_cat      8.692678e-01
robbery_cat     9.624259e-02
pop            7.517475e-100
dtype: float64

Neither murder_cat or robbery_cat are statistically significant in this model.

## Conclusions from cross validation

Model 1 performs the best in terms of explained variance, both overall and through cross validation. It also has the most statistically significant terms. This model is therefore the one chosen to move onto the next stage - validation against another data set.